In [1]:
import cPickle
import networkx as nx
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
path='../input/'
train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')
questions = pd.read_csv(path+'question.csv')

In [3]:
train = pd.merge(train, questions, left_on=['q1'], right_on=['qid'], how='left')
train = train.rename(columns={'words': 'q1_words', 'chars': 'q1_chars'})
del train['qid']
train = pd.merge(train, questions, left_on=['q2'], right_on=['qid'], how='left')
train = train.rename(columns={'words': 'q2_words', 'chars': 'q2_chars'})
train.drop(['q1', 'q2', 'qid'], axis=1, inplace=True)

In [4]:
train.head()

,label,q1_words,q1_chars,q2_words,q2_chars
0,1,W04465 W04058 W05284 W02916,L2218 L2568 L0360 L0242 L2218 L0741,W18238 W18843 W01490 W09905,L3019 L0104 L0582 L2218 L1861 L1556 L0242
1,0,W10054 W04476 W09996 W12244 W18103,L2376 L2168 L0050 L1187 L0104 L2432 L0902 L014...,W18439 W00863 W04259 W00740 W16070,L0156 L2452 L1187 L0104 L2459 L2979 L2613 L0449
2,0,W04346 W17378 W19355 W17926 W14185 W11567 W07863,L2323 L1526 L2214 L1132 L2723 L1861 L2249 L050...,W14586 W09745 W06017 W09067 W16319,L2568 L0971 L1291 L0358 L0037 L2582
3,0,W17508 W09996 W19662 W17534 W11399 W17057 W182...,L0018 L2321 L1346 L2432 L0902 L1149 L1980 L187...,W18238 W02357 W06606,L3019 L0104 L1104 L1935 L1683 L2495 L2812
4,0,W13157 W03390 W01952 W05789 W17378 W08714 W13157,L2271 L1346 L1389 L2932 L0466 L2218 L1971 L221...,W04476 W06606 W00316 W13157,L0050 L1187 L0104 L1683 L2495 L2812 L1588 L255...


In [5]:
test = pd.merge(test, questions, left_on=['q1'], right_on=['qid'], how='left')
test = test.rename(columns={'words': 'q1_words', 'chars': 'q1_chars'})
del test['qid']
test = pd.merge(test, questions, left_on=['q2'], right_on=['qid'], how='left')
test = test.rename(columns={'words': 'q2_words', 'chars': 'q2_chars'})
test.drop(['q1', 'q2', 'qid'], axis=1, inplace=True)

In [6]:
test.head()

,q1_words,q1_chars,q2_words,q2_chars
0,W17342 W06766 W04049 W16319 W07863,L0358 L0143 L0942 L1872 L1236 L3046 L0055 L258...,W11668 W17378 W11399 W14113,L1791 L2214 L1872 L1236 L0947 L2323
1,W17378 W17534 W02000 W19499 W05016 W06441 W193...,L2214 L1980 L1526 L2669 L0590 L2812 L0549 L000...,W10157 W13631 W11299 W07863,L1796 L2568 L0127 L0004 L0030 L2120 L2927
2,W17378 W19355 W03914 W18238 W03746,L2214 L1132 L2292 L0158 L3019 L0104 L0156 L0762,W19468 W10157 W02288 W18951 W18448,L2253 L1796 L2568 L0156 L0762 L1486 L2292 L1759
3,W16938 W17867 W18843 W01490 W06036,L2172 L1074 L0582 L2218 L1861 L2705 L1037,W01490 W00496,L2218 L1861 L0377 L2619
4,W17378 W19355 W01490 W11030 W19349 W18399 W03390,L2214 L1132 L2218 L1861 L1536 L0146 L0607 L186...,W04745 W04622 W00863 W16521 W17765 W06502 W18050,L2572 L0135 L0562 L0445 L1187 L0104 L2705 L125...


In [7]:
y_train=train.label.values

# max_clique_features

In [73]:
data=pd.concat([train[['q1_words','q2_words']],test[['q1_words','q2_words']]]).reset_index()
data.drop('index',axis=1,inplace=True)

In [75]:
q_list={}
dd=data.values
for i in tqdm(np.arange(data.values.shape[0])):
#for i in np.arange(dd.shape[0]):
    q1,q2=dd[i]
    if q_list.setdefault(q1,[i])!=[i]:
        q_list[q1].append(i)
    if q_list.setdefault(q2,[i])!=[i]:
        q_list[q2].append(i)
data['q1_words_link']=data.q1_words.map(q_list)
data['q2_words_link']=data.q2_words.map(q_list)

100%|██████████| 427342/427342 [00:01<00:00, 244583.96it/s]


In [76]:
data['adj_node']=data.apply(lambda x:x['q1_words_link']+x['q2_words_link'],axis=1)

In [77]:
G=nx.Graph()
cnt=0
for i,adj_list in tqdm(enumerate(data.adj_node.values)):
#for i,adj_list in enumerate(data.adj_node.values):
    edges=[(i,item) for item in adj_list if item !=i]
    if edges==[]:
#        if cnt%100000==0:
#            print("ignore cnts %d" %cnt)
        cnt+=1
    else:
        G.add_edges_from(edges)  
print("ignore cnts %d" %cnt)

427342it [00:36, 11572.19it/s]

ignore cnts 106343


In [78]:
print('begin train')
cnt=0
max_clique=np.zeros(data.shape[0])
for clique in tqdm(nx.find_cliques(G)):
#for clique in nx.enumerate_all_cliques(G):
    if cnt%100000==0:
        print("deal cnts %d" %cnt)
    len_clique=len(clique)
    for item in clique:
        max_clique[item]=max(max_clique[item],len_clique)
    cnt+=1


print("totally max_clique %d" %cnt)

pd.to_pickle(max_clique,path+'max_clique.pkl')

0it [00:00, ?it/s]

begin train


40it [00:09,  6.69s/it]

deal cnts 0


101051it [02:50, 7950.81it/s]

deal cnts 100000


117286it [02:53, 676.53it/s] 


totally max_clique 117286


In [80]:
max_clique.shape

(427342,)

In [82]:
train.shape[0], test.shape[0]

(254386, 172956)

In [85]:
train_max_clique_features = max_clique[:train.shape[0]]
test_max_clique_features = max_clique[train.shape[0]:]

In [88]:
train_features = pd.DataFrame({'id': np.arange(train.shape[0]),
                               'max_clique': train_max_clique_features})
test_features = pd.DataFrame({'id': np.arange(test.shape[0]),
                               'max_clique': test_max_clique_features})

In [90]:
train_features.head()

,id,max_clique
0,0,76.0
1,1,0.0
2,2,2.0
3,3,2.0
4,4,2.0


In [91]:
with open('../input/features/train_max_clique_features.pkl', "wb") as f:
    cPickle.dump(train_features, f, -1)

with open('../input/features/test_max_clique_features.pkl', "wb") as f:
    cPickle.dump(test_features, f, -1)

# TFIDF features

In [92]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

In [93]:
train.head()

,label,q1_words,q1_chars,q2_words,q2_chars
0,1,W04465 W04058 W05284 W02916,L2218 L2568 L0360 L0242 L2218 L0741,W18238 W18843 W01490 W09905,L3019 L0104 L0582 L2218 L1861 L1556 L0242
1,0,W10054 W04476 W09996 W12244 W18103,L2376 L2168 L0050 L1187 L0104 L2432 L0902 L014...,W18439 W00863 W04259 W00740 W16070,L0156 L2452 L1187 L0104 L2459 L2979 L2613 L0449
2,0,W04346 W17378 W19355 W17926 W14185 W11567 W07863,L2323 L1526 L2214 L1132 L2723 L1861 L2249 L050...,W14586 W09745 W06017 W09067 W16319,L2568 L0971 L1291 L0358 L0037 L2582
3,0,W17508 W09996 W19662 W17534 W11399 W17057 W182...,L0018 L2321 L1346 L2432 L0902 L1149 L1980 L187...,W18238 W02357 W06606,L3019 L0104 L1104 L1935 L1683 L2495 L2812
4,0,W13157 W03390 W01952 W05789 W17378 W08714 W13157,L2271 L1346 L1389 L2932 L0466 L2218 L1971 L221...,W04476 W06606 W00316 W13157,L0050 L1187 L0104 L1683 L2495 L2812 L1588 L255...


In [94]:
def get_distinct_q1(row):
    q1words = {}
    q2words = {}
    for word in str(row['q1_words']).lower().split():
        q1words[word] = 1
    for word in str(row['q2_words']).lower().split():
        q2words[word] = 1
    shared_words_in_q1 = [w for w in q1words.keys() if w not in q2words]
    #shared_words_in_q2 = [w for w in q2words.keys() if w not in q1words]
    if len(shared_words_in_q1)==0:
        shared_words_in_q1=['#']
    #if len(shared_words_in_q2)==0:
    #    shared_words_in_q2=['#']
    #R = (len(shared_words_in_q1) + len(shared_words_in_q2))*1.0/(len(q1words) + len(q2words))
    return ' '.join(shared_words_in_q1)

In [95]:
def get_distinct_q2(row):
    q1words = {}
    q2words = {}
    for word in str(row['q1_words']).lower().split():
        q1words[word] = 1
    for word in str(row['q2_words']).lower().split():
        q2words[word] = 1
    #shared_words_in_q1 = [w for w in q1words.keys() if w not in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w not in q1words]
    #if len(shared_words_in_q1)==0:
    #    shared_words_in_q1=['#']
    if len(shared_words_in_q2)==0:
        shared_words_in_q2=['#']
    #R = (len(shared_words_in_q1) + len(shared_words_in_q2))*1.0/(len(q1words) + len(q2words))
    return ' '.join(shared_words_in_q2)

In [96]:
train['distinct_porter_q1']=train.apply(get_distinct_q1,axis=1)
train['distinct_porter_q2']=train.apply(get_distinct_q2,axis=1)
test['distinct_porter_q1']=test.apply(get_distinct_q1,axis=1)
test['distinct_porter_q2']=test.apply(get_distinct_q2,axis=1)

In [97]:
train[['distinct_porter_q1','distinct_porter_q2']].to_csv(path+'train_distinct.csv',index=False)
test[['distinct_porter_q1','distinct_porter_q2']].to_csv(path+'test_distinct.csv',index=False)

In [98]:
corpus=[]
for fe in ['distinct_porter_q1','distinct_porter_q2']:
    corpus+=train[fe].astype(str).values.tolist()

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

In [102]:
tfidf__norm = "l2"
tfidf__max_df = 0.75
tfidf__min_df = 3
tfidf=TfidfVectorizer(ngram_range=(1,1), min_df=tfidf__min_df,max_df=tfidf__max_df,norm=tfidf__norm)
tfidf.fit(corpus)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.75, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [110]:
pd.to_pickle(tfidf.transform(train.distinct_porter_q1),path+'train_distinct_porter_q1_tfidf.pkl')
pd.to_pickle(tfidf.transform(train.distinct_porter_q2),path+'train_distinct_porter_q2_tfidf.pkl')
pd.to_pickle(tfidf.transform(test.distinct_porter_q1),path+'test_distinct_porter_q1_tfidf.pkl')
pd.to_pickle(tfidf.transform(test.distinct_porter_q2),path+'test_distinct_porter_q2_tfidf.pkl')

In [115]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
seed = 1024
np.random.seed(seed)
path = "../input/"

def calc_cosine_dist(text_a ,text_b):
    return pairwise_distances(text_a, text_b, metric='cosine')[0][0]

In [116]:
print('Tfidf distinct Similarity part')
train_question1_distinct_tfidf = pd.read_pickle(path+'train_distinct_porter_q1_tfidf.pkl')
test_question1_distinct_tfidf = pd.read_pickle(path+'test_distinct_porter_q1_tfidf.pkl')
train_question2_distinct_tfidf = pd.read_pickle(path+'train_distinct_porter_q2_tfidf.pkl')
test_question2_distinct_tfidf = pd.read_pickle(path+'test_distinct_porter_q2_tfidf.pkl')

Tfidf distinct Similarity part


In [117]:
train_tfidf_sim = []
for r1,r2 in zip(train_question1_distinct_tfidf,train_question2_distinct_tfidf):
    train_tfidf_sim.append(calc_cosine_dist(r1,r2))
test_tfidf_sim = []
for r1,r2 in zip(test_question1_distinct_tfidf,test_question2_distinct_tfidf):
    test_tfidf_sim.append(calc_cosine_dist(r1,r2))
train_tfidf_sim = np.array(train_tfidf_sim)
test_tfidf_sim = np.array(test_tfidf_sim)

In [119]:
train_features = pd.DataFrame({'id': np.arange(train.shape[0]),
                               'tfidf_sim': train_tfidf_sim})
test_features = pd.DataFrame({'id': np.arange(test.shape[0]),
                               'tfidf_sim': test_tfidf_sim})

In [120]:
with open('../input/features/train_tfidf_sim_features.pkl', "wb") as f:
    cPickle.dump(train_features, f, -1)

with open('../input/features/test_tfidf_sim_features.pkl', "wb") as f:
    cPickle.dump(test_features, f, -1)

# doc2vec

In [9]:
import pandas as pd
import numpy as np
import os
import string
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis


In [10]:
train.head()

,label,q1_words,q1_chars,q2_words,q2_chars
0,1,W04465 W04058 W05284 W02916,L2218 L2568 L0360 L0242 L2218 L0741,W18238 W18843 W01490 W09905,L3019 L0104 L0582 L2218 L1861 L1556 L0242
1,0,W10054 W04476 W09996 W12244 W18103,L2376 L2168 L0050 L1187 L0104 L2432 L0902 L014...,W18439 W00863 W04259 W00740 W16070,L0156 L2452 L1187 L0104 L2459 L2979 L2613 L0449
2,0,W04346 W17378 W19355 W17926 W14185 W11567 W07863,L2323 L1526 L2214 L1132 L2723 L1861 L2249 L050...,W14586 W09745 W06017 W09067 W16319,L2568 L0971 L1291 L0358 L0037 L2582
3,0,W17508 W09996 W19662 W17534 W11399 W17057 W182...,L0018 L2321 L1346 L2432 L0902 L1149 L1980 L187...,W18238 W02357 W06606,L3019 L0104 L1104 L1935 L1683 L2495 L2812
4,0,W13157 W03390 W01952 W05789 W17378 W08714 W13157,L2271 L1346 L1389 L2932 L0466 L2218 L1971 L221...,W04476 W06606 W00316 W13157,L0050 L1187 L0104 L1683 L2495 L2812 L1588 L255...


In [11]:
data=[]
for fe in ['q1_words','q2_words']:
    data.extend(train[fe].astype(str).tolist())
    data.extend(test[fe].astype(str).tolist())

f = open('all_sentences.txt','w')
for i,line in enumerate(data):
    f.write('_*'+str(i)+' '+line+'\n')
f.close()

In [12]:
!git clone https://github.com/mesnilgr/iclr15.git
!cp iclr15/scripts/word2vec.c .
!gcc word2vec.c -o word2vec -lm -pthread -O3 -march=native -funroll-loops

!shuf all_sentences.txt > all_sentences_shuf.txt
!time ./word2vec -train all_sentences.txt -output vectors.txt -cbow 0 -size 100 -window 10 -negative 5 -hs 1 -sample 1e-3 -threads 12 -binary 0 -iter 20 -min-count 1 -sentence-vectors 1

!grep '_\*' vectors.txt | sed -e 's/_\*//' | sort -n > sentence_vectors.txt

fatal: destination path 'iclr15' already exists and is not an empty directory.
word2vec.c: In function ‘ReadVocab’:
word2vec.c:320:5: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunused-result]
     fscanf(fin, "%lld%c", &vocab[a].cn, &c);
     ^
Starting training using file all_sentences.txt
Vocab size: 870565
Words in train file: 6945641
Alpha: 0.000010  Progress: 99.97%  Words/thread/sec: 47.56k  2939.74user 24.93system 6:06.93elapsed 807%CPU (0avgtext+0avgdata 1782344maxresident)k
0inputs+1625384outputs (0major+365376minor)pagefaults 0swaps


In [13]:
!cat sentence_vectors.txt | wc -l

854684


In [15]:
854684 / 2

427342

In [16]:
train.shape[0] + test.shape[0]

427342

In [17]:
!head -n 427342  sentence_vectors.txt > sentence_vectors_q1.txt
!tail -n 427342  sentence_vectors.txt > sentence_vectors_q2.txt

In [18]:
f1 = open('sentence_vectors_q1.txt','r')
f2 = open('sentence_vectors_q2.txt','r')
_cosine=[]
_cityblock=[]
_jaccard=[]
_canberra=[]
_euclidean=[]
_minkowski=[]
_braycurtis=[]
for q1 in f1:
    q2 = f2.readline()
    v1=np.array(q1.strip().split()[1:]).astype(float)
    v2=np.array(q2.strip().split()[1:]).astype(float)
    #cos=np.dot(v1,v2)/np.sqrt((np.dot(v1,v1)*np.dot(v2,v2)))
    #abs_dis=np.sum(np.abs(v1-v2))
    _cosine.append(cosine(v1,v2))
    _cityblock.append(cityblock(v1,v2))
    _jaccard.append(jaccard(v1,v2))
    _canberra.append(canberra(v1,v2))
    _euclidean.append(euclidean(v1,v2))
    _minkowski.append(minkowski(v1,v2,3))
    _braycurtis.append(braycurtis(v1,v2))
f1.close()
f2.close()

In [19]:
doc_sim=np.vstack([_cosine,
           _cityblock,
           _jaccard,
           _canberra,
           _euclidean,
           _minkowski,
           _braycurtis]).T

In [20]:
doc_sim.shape

(427342, 7)

In [21]:
train_doc_sim = doc_sim[:y_train.shape[0]]
test_doc_sim = doc_sim[y_train.shape[0]:]

In [31]:
feature_names = ['_cosine',
           '_cityblock',
           '_jaccard',
           '_canberra',
           '_euclidean',
           '_minkowski',
           '_braycurtis']

In [35]:
train_doc_sim_features = pd.DataFrame(train_doc_sim, columns=feature_names)
test_doc_sim_features = pd.DataFrame(test_doc_sim, columns=feature_names)

In [36]:
train_doc_sim_features['id'] = np.arange(train.shape[0])
test_doc_sim_features['id'] = np.arange(test.shape[0])

In [38]:
with open('../input/features/train_doc_sim_features.pkl', "wb") as f:
    cPickle.dump(train_doc_sim_features, f, -1)

with open('../input/features/test_doc_sim_features.pkl', "wb") as f:
    cPickle.dump(test_doc_sim_features, f, -1)

# lda_feature

In [23]:
import pandas as pd
import scipy.sparse as ssp
import numpy as np
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
warnings.filterwarnings(action='ignore')
from sklearn.decomposition import LatentDirichletAllocation

In [24]:
ft = ['q1_words','q2_words']

len_train = train.shape[0]
lda = LatentDirichletAllocation(n_topics=20, doc_topic_prior=None, 
                                topic_word_prior=None, learning_method='batch', 
                                learning_decay=0.7, learning_offset=10.0, max_iter=10,
                                batch_size=128, evaluate_every=-1, total_samples=1000000.0, 
                                perp_tol=0.1, mean_change_tol=0.001, max_doc_update_iter=100,
                                n_jobs=16, verbose=0, random_state=42)
bow = CountVectorizer(ngram_range=(1,1),max_df=0.95,min_df=3,stop_words='english')
vect_orig = make_pipeline(bow,lda)

corpus = []
for f in ft:
    train[f] = train[f].astype(str)
    test[f] = test[f].astype(str)
    corpus+=train[f].values.tolist()

vect_orig.fit(
    corpus
    )



Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.95, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None, stop_words='eng...          random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0))])

In [39]:
for f in ft:
    train_lda = vect_orig.transform(train[f].values.tolist())
    test_lda = vect_orig.transform(test[f].values.tolist())

print train_lda.shape,test_lda.shape

(254386, 20) (172956, 20)


In [40]:
train_lda_features = pd.DataFrame(train_lda, columns=['lda_{}'.format(i) for i in range(test_lda.shape[1])])
test_lda_features = pd.DataFrame(test_lda, columns=['lda_{}'.format(i) for i in range(test_lda.shape[1])])

In [42]:
train_lda_features['id'] = np.arange(train.shape[0])
test_lda_features['id'] = np.arange(test.shape[0])

In [43]:
with open('../input/features/train_lda_features.pkl', "wb") as f:
    cPickle.dump(train_lda_features, f, -1)

with open('../input/features/test_lda_features.pkl', "wb") as f:
    cPickle.dump(test_lda_features, f, -1)

# hash sentence

In [52]:
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt
from random import random,shuffle
import pickle
import sys
import string
import random
seed =1024
random.seed(seed)

train_hash = pd.DataFrame()
train_hash['question1_hash'] = train['q1_words'].map(lambda x : hash(x))
train_hash['question2_hash'] = train['q2_words'].map(lambda x : hash(x))

test_hash = pd.DataFrame()
test_hash['question1_hash'] = test['q1_words'].map(lambda x : hash(x))
test_hash['question2_hash'] = test['q2_words'].map(lambda x : hash(x))

train_hash.to_csv('train_hashed.csv',index=False)
test_hash.to_csv('test_hashed.csv',index=False)

# pagerank_directed

In [53]:
import networkx as nx
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from csv import DictReader

In [54]:
def prepare_graph(paths):
    G = nx.DiGraph()

    idf_dict = dict()
    for path in paths:
        print(path)
        c = 0
        start = datetime.now()

        for t, row in enumerate(DictReader(open(path), delimiter=',')):
            if c%100000==0:
                print 'finished',c
            q1 = str(row['question1_hash'])
            q2 = str(row['question2_hash'])
            G.add_edge(q1,q2)
            c+=1
    end = datetime.now()
    print('times:',end-start)
    return G

In [55]:
path = './'
G = prepare_graph([path+'train_hashed.csv', path+'test_hashed.csv'])

./train_hashed.csv
finished 0
finished 100000
finished 200000
./test_hashed.csv
finished 0
finished 100000
('times:', datetime.timedelta(0, 2, 596494))


In [56]:
max_clique=nx.pagerank(G,alpha=0.9)

def prepare_hash_clique_stats(path,out,idf_dict):

    print path
    c = 0
    start = datetime.now()
    header = [                
        'min_pr',
        'max_pr',
        ]
    header = ','.join(header)
    with open(out, 'w') as outfile:
        outfile.write('%s\n'%header)
        for t, row in enumerate(DictReader(open(path), delimiter=',')): 
            if c%100000==0:
                print 'finished',c
            q1 = str(row['question1_hash'])
            q2 = str(row['question2_hash'])
            # q1 = hash(q1)
            # q2 = hash(q2)
            
            q1_idf = idf_dict.get(q1,0.0)
            q2_idf = idf_dict.get(q2,0.0)
            
            max_q1 = max((q1_idf,q2_idf))
            min_q1 = min((q1_idf,q2_idf))


            outfile.write('%s,%s\n' % (
                max_q1,
                min_q1,
                ))
            
            c+=1
            end = datetime.now()


    print 'times:',end-start

In [57]:
prepare_hash_clique_stats(path+'train_hashed.csv', path+'train_pagerank_directed.csv', max_clique)

./train_hashed.csv
finished 0
finished 100000
finished 200000
times: 0:00:01.967167


In [58]:
prepare_hash_clique_stats(path+'test_hashed.csv',path+'test_pagerank_directed.csv',max_clique)

./test_hashed.csv
finished 0
finished 100000
times: 0:00:01.288408


In [59]:
train_pagerank_directed = pd.read_csv('train_pagerank_directed.csv')
test_pagerank_directed = pd.read_csv('test_pagerank_directed.csv')

In [61]:
train_pagerank_directed.shape

(254386, 2)

In [63]:
train_pagerank_directed['id'] = np.arange(train.shape[0])
test_pagerank_directed['id'] = np.arange(test.shape[0])

with open('../input/features/train_pagerank_directed_features.pkl', "wb") as f:
    cPickle.dump(train_pagerank_directed, f, -1)

with open('../input/features/test_pagerank_directed_features.pkl', "wb") as f:
    cPickle.dump(test_pagerank_directed, f, -1)

In [65]:
data=pd.concat([train[['q1_words','q2_words']],test[['q1_words','q2_words']]]).reset_index()
data.drop('index',axis=1,inplace=True)
q_list={}
dd=data.values
for i in tqdm(np.arange(data.values.shape[0])):
#for i in np.arange(dd.shape[0]):
    q1,q2=dd[i]
    if q_list.setdefault(q1,[i])!=[i]:
        q_list[q1].append(i)
    if q_list.setdefault(q2,[i])!=[i]:
        q_list[q2].append(i)
data['question1_link']=data.q1_words.map(q_list)
data['question2_link']=data.q2_words.map(q_list)

100%|██████████| 427342/427342 [00:02<00:00, 158585.76it/s]


In [66]:
data['adj_node']=data.apply(lambda x:x['question1_link']+x['question2_link'],axis=1)

In [67]:
print('train')

G=nx.Graph()
cnt=0
for i,adj_list in tqdm(enumerate(data.adj_node.values)):
    edges=[(i,item) for item in adj_list if item !=i]
    if edges==[]:
        if cnt%100000==0:
            print("ignore cnts %d" %cnt)
            cnt+=1
    else:
        G.add_edges_from(edges)  


# In[ ]:

pr=nx.pagerank(G)

train


1101it [00:00, 11008.15it/s]

ignore cnts 0


427342it [00:39, 10927.81it/s]


In [70]:
len(pr)

320999

In [76]:
pd.to_pickle(pr,path+'page_rank.pkl')

# wordnet

In [72]:
import string
import numpy as np
import pandas as pd
from nltk.corpus import wordnet as wn

def basic_token(text):
    return list(text.strip().lower().split())

class WordNet_Similarity():
    def __init__(self, metric="path",double_aggregator=False):
        """
        :param metric: path lch and wup metric
        :param double_aggregator:
        """
        self.metric = metric
        self.aggregation_mode_prev = ["mean", "max", "median"]
        self.aggregation_mode = ["mean", "std", "max", "min", "median"]
        self.aggregator = [None if m == "" else getattr(np, m) for m in self.aggregation_mode]
        self.aggregator_prev = [None if m == "" else getattr(np, m) for m in self.aggregation_mode_prev]
        self.double_aggregator = double_aggregator
        if self.metric == "path":
            self.metric_func = lambda syn1, syn2: wn.path_similarity(syn1, syn2)
        elif self.metric == "lch":
            self.metric_func = lambda syn1, syn2: wn.lch_similarity(syn1, syn2)
        elif self.metric == "wup":
            self.metric_func = lambda syn1, syn2: wn.wup_similarity(syn1, syn2)
        else:
            raise (ValueError("Wrong similarity metric: %s, should be one of path/lch/wup." % self.metric))

    def _maximum_similarity_for_two_synset_list(self, syn_list1, syn_list2):
        s = 0.
        if syn_list1 and syn_list2:
            for syn1 in syn_list1:
                for syn2 in syn_list2:
                    try:
                        _s = self.metric_func(syn1, syn2)
                    except:
                        _s = MISSING_VALUE_NUMERIC
                    if _s and _s > s:
                        s = _s
        return s

    def transform_one(self, obs, target):
        obs_tokens = basic_token(obs)
        target_tokens = basic_token(target)
        obs_synset_list = [wn.synsets(obs_token) for obs_token in obs_tokens]
        target_synset_list = [wn.synsets(target_token) for target_token in target_tokens]
        val_list = []
        for obs_synset in obs_synset_list:
            _val_list = []
            for target_synset in target_synset_list:                     #obs_synset maxmetric
                _s = self._maximum_similarity_for_two_synset_list(obs_synset, target_synset)
                _val_list.append(_s)
            if len(_val_list) == 0:
                _val_list = [MISSING_VALUE_NUMERIC]
            val_list.append(_val_list)
        if len(val_list) == 0:
            val_list = [[MISSING_VALUE_NUMERIC]]
        return val_list

    def fit_transform(self,data_all):
        """
        :param data_all: the numpy fields dim 0 is obs dim1 is target
        :return: sim
        """
        score = list(map(WN.transform_one, data_all[:, 0], data_all[:, 1]))
        self.N = data_all.shape[0]
        if self.double_aggregator:
            res = np.zeros((self.N, len(self.aggregator_prev) * len(self.aggregator)), dtype=float)
            for m, aggregator_prev in enumerate(self.aggregator_prev):
                for n, aggregator in enumerate(self.aggregator):
                    idx = m * len(self.aggregator) + n
                    for i in range(self.N):
                        # process in a safer way
                        try:
                            tmp = []
                            for l in score[i]:
                                try:
                                    s = aggregator_prev(l)
                                except:
                                    s = MISSING_VALUE_NUMERIC
                                tmp.append(s)
                        except:
                            tmp = [MISSING_VALUE_NUMERIC]
                        try:
                            s = aggregator(tmp)
                        except:
                            s = MISSING_VALUE_NUMERIC
                        res[i, idx] = s
        else:
            res = np.zeros((self.N, len(self.aggregator)), dtype=float)
            for m, aggregator in enumerate(self.aggregator):
                for i in range(self.N):
                    # process in a safer way
                    try:
                        s = aggregator(score[i])
                    except:
                        s = MISSING_VALUE_NUMERIC
                    res[i, m] = s
        return res

class WordNet_Path_Similarity(WordNet_Similarity):
    def __init__(self):
        super(WordNet_Path_Similarity, self).__init__("path")

class WordNet_Lch_Similarity(WordNet_Similarity):
    def __init__(self):
        super(WordNet_Lch_Similarity, self).__init__( "lch")

class WordNet_Wup_Similarity(WordNet_Similarity):
    def __init__(self):
        super(WordNet_Wup_Similarity, self).__init__( "wup")

In [73]:
np.random.seed(1024)
path = '../input/'

MISSING_VALUE_NUMERIC = -1.
TRAIN_DATA_FILE = path+'train_clean.pkl'
TEST_DATA_FILE = path+ 'test_clean.pkl'
ft = ['q1_words','q2_words']

train_data = train[ft]
test_data = test[ft]
data_all = np.vstack([train_data,test_data])

In [74]:
WN = WordNet_Similarity(double_aggregator=True)

In [75]:
WordNetSim = WN.fit_transform(data_all)

In [78]:
train_wordnet = WordNetSim[:train_data.shape[0]]
test_wordnet = WordNetSim[train_data.shape[0]:]

In [79]:
train_wordnet.shape, test_wordnet.shape

((254386, 15), (172956, 15))

In [81]:
train_wordnet = pd.DataFrame(train_wordnet, columns=['lda_{}'.format(i) for i in range(train_wordnet.shape[1])])
test_wordnet = pd.DataFrame(test_wordnet, columns=['lda_{}'.format(i) for i in range(test_wordnet.shape[1])])

train_wordnet['id'] = np.arange(train.shape[0])
test_wordnet['id'] = np.arange(test.shape[0])

with open('../input/features/train_wordnet_features.pkl', "wb") as f:
    cPickle.dump(train_wordnet, f, -1)

with open('../input/features/test_wordnet_features.pkl', "wb") as f:
    cPickle.dump(test_wordnet, f, -1)

# network_feature

In [77]:
import networkx as nx
import numpy as np
import pandas as pd
import scipy.stats as sps
from tqdm import tqdm
import matplotlib.pyplot as plt

In [85]:
pr=pd.read_pickle('../input/page_rank.pkl')                                         ###node as index
clique=pd.read_pickle('../input/max_clique.pkl')                                    ###node as index

In [86]:
train_hashed=pd.read_csv('train_hashed.csv')
test_hashed=pd.read_csv('test_hashed.csv')

In [89]:
train_hashed['pr_max']=train_hashed[['question1_hash','question2_hash']].values.max(axis=1)
test_hashed['pr_max']=test_hashed[['question1_hash','question2_hash']].values.max(axis=1)
train_hashed['pr_min']=train_hashed[['question1_hash','question2_hash']].values.min(axis=1)
test_hashed['pr_min']=test_hashed[['question1_hash','question2_hash']].values.min(axis=1)

In [90]:
train_hashed['pr_max_ratio_pr_min']=train_hashed.pr_max/train_hashed.pr_min
train_hashed['pr_dis']=train_hashed.pr_max-train_hashed.pr_min
train_hashed['pr_sum']=train_hashed.pr_max+train_hashed.pr_min

In [91]:
test_hashed['pr_max_ratio_pr_min']=test_hashed.pr_max/test_hashed.pr_min
test_hashed['pr_dis']=test_hashed.pr_max-test_hashed.pr_min
test_hashed['pr_sum']=test_hashed.pr_max+test_hashed.pr_min

In [93]:
train_hashed.head()

,question1_hash,question2_hash,pr_max,pr_min,pr_max_ratio_pr_min,pr_dis,pr_sum
0,-4758480463380089892,8245053456646392776,8245053456646392776,-4758480463380089892,-1.732707,-5443210153683068948,3486572993266302884
1,-6157910120044557781,-4208899837926650002,-4208899837926650002,-6157910120044557781,0.683495,1949010282117907779,8079934115738343833
2,-8184860297383627901,-8227147811250115192,-8184860297383627901,-8227147811250115192,0.994860,42287513866487291,2034735965075808523
3,-2815498668808458623,-3878704118046036344,-2815498668808458623,-3878704118046036344,0.725886,1063205449237577721,-6694202786854494967
4,-3902575321630500694,-471978648261078005,-471978648261078005,-3902575321630500694,0.120940,3430596673369422689,-4374553969891578699


In [95]:
train_hashed.shape, test_hashed.shape

((254386, 7), (172956, 7))

In [96]:
pr_features=np.zeros(train.shape[0]+test.shape[0])
for key,val in pr.items():
    pr_features[key]=val

In [97]:
train_pr_features = pr_features[:train.shape[0]]
test_pr_features = pr_features[train.shape[0]:]

In [98]:
train_pr_features.shape, test_pr_features.shape

((254386,), (172956,))

In [101]:
train_hashed = pd.DataFrame(train_hashed, columns=['hashed_{}'.format(i) for i in range(train_hashed.shape[1])])
test_hashed = pd.DataFrame(test_hashed, columns=['hashed_{}'.format(i) for i in range(test_hashed.shape[1])])

train_hashed['id'] = np.arange(train.shape[0])
test_hashed['id'] = np.arange(test.shape[0])

with open('../input/features/train_hashed_features.pkl', "wb") as f:
    cPickle.dump(train_hashed, f, -1)

with open('../input/features/test_hashed_features.pkl', "wb") as f:
    cPickle.dump(test_hashed, f, -1)

In [102]:
train_pr_features = pd.DataFrame(train_pr_features, columns=['pr_feature'])
test_pr_features = pd.DataFrame(test_pr_features, columns=['pr_feature'])

train_pr_features['id'] = np.arange(train.shape[0])
test_pr_features['id'] = np.arange(test.shape[0])

with open('../input/features/train_pr_features.pkl', "wb") as f:
    cPickle.dump(train_pr_features, f, -1)

with open('../input/features/test_pr_features.pkl', "wb") as f:
    cPickle.dump(test_pr_features, f, -1)